# Multiplying upper triangular matrices

This notebook helps you implement the operation $ C := U R $ where $ C, U, R \in \mathbb{R}^{n \times n} $, and $ U $ and $ R $ are upper triangular.  $ U $ and $ R $ are stored in the upper triangular part of numpy matrices <code> U </code> and <code> R </code>. The upper triangular part of matrix <code> C </code> is to be overwritten with the resulting upper triangular matrix.  

First, we create some matrices.

In [1]:
n = 5

C = rand(n, n)
println( "C = " )
C

C = 


5×5 Array{Float64,2}:
 0.363805  0.470377  0.304103  0.460884  0.0186091
 0.794673  0.941969  0.198176  0.604494  0.700519 
 0.641871  0.509722  0.272848  0.947105  0.477244 
 0.55827   0.182727  0.179498  0.551514  0.382309 
 0.599656  0.681311  0.218425  0.182557  0.418006 

In [2]:
Cold = copy( C ) # an alternative way of doing a "hard" copy, in this case of a matrix
    
U = rand(n, n)
println( "U = " )
U

U = 


5×5 Array{Float64,2}:
 0.0200118  0.24006    0.469801  0.0260589  0.828365 
 0.547712   0.0565943  0.700089  0.954994   0.493988 
 0.136371   0.838578   0.176311  0.709253   0.0337659
 0.962615   0.48762    0.950478  0.847303   0.0424887
 0.246439   0.732695   0.701028  0.548225   0.502472 

In [3]:
R = rand(n, n)
println( "R = " )
R

R = 


5×5 Array{Float64,2}:
 0.130688   0.954879  0.387661  0.561678   0.316197
 0.180876   0.435833  0.71329   0.0807493  0.649268
 0.0879662  0.432293  0.481885  0.511855   0.667453
 0.538766   0.486865  0.384126  0.459313   0.234974
 0.888315   0.142085  0.442179  0.649281   0.440306

## <h2>The algorithm  </h2>  <image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/5_5_1_10_Answer.png" alt="Upper triangular matrix-matrix multiplication" width="80%">

<h2> The routine <code> Trtrmm_uu_unb_var1!( U, R, C ) </code> </h2>

This routine computes $ C := U R + C $.  The "\_uu\_" means that $ U $ and $ R $ are upper triangular matrices (which means $ C $ is too).  However, the lower triangular parts of numpy matrices <code> U </code>, <code> R </code>, and <code> C </code> are not to be "touched".
    
The specific laff function you will want to use is some subset of
<ul>
<li> <code> laff.gemv!( trans, alpha, A, x, beta, y ) </code> which computes $ y := \alpha A x + \beta y $ or $ y := \alpha A^T x + \beta y $ depending on whether <code> trans = 'No transpose' </code> or <code> trans = 'Transpose' </code> </li>
    <li> <code> laff.ger!( alpha, x, y, A ) </code> which computes the rank-1 update (adds a multiple of an outer product to a matrix)
$ A := \alpha x y^T + A $. </li>
    <li> <code> laff.axpy!( alpha, x, y ) </code></li>
    <li>    <code> laff.dots!( x, y, alpha ) </code></li>
</ul>

<font color=red> Hint:</font>  If you multiply with $ U_{00} $, you will want to use <code> UpperTriangular( U00 ) </code> to make sure you don't compute with the nonzeroes below the diagonal.

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [6]:
using LinearAlgebra

In [11]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trtrmm_uu_unb_var1!(U, R, C)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    RTL, RTR, 
    RBL, RBR  = flame.part_2x2(R, 
                               0, 0, "TL")

    CTL, CTR, 
    CBL, CBR  = flame.part_2x2(C, 
                               0, 0, "TL")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        R00,  r01,   R02,  
        r10t, rho11, r12t, 
        R20,  r21,   R22   = flame.repart_2x2_to_3x3(RTL, RTR, 
                                                     RBL, RBR, 
                                                     1, 1, "BR")

        C00,  c01,     C02,  
        c10t, gamma11, c12t, 
        C20,  c21,     C22   = flame.repart_2x2_to_3x3(CTL, CTR, 
                                                       CBL, CBR, 
                                                       1, 1, "BR")

        #------------------------------------------------------------#

        # imporant:  You do not need to recompute C00 = U00 * R00!!!!
        myU00 = copy(U00)
        m, n = size(myU00)
        for i in 1:m
            for j in 1:(i - 1)
                myU00[i, j] = 0.0
            end
        end
        laff.gemv!( "No transpose", 1.0, U00, r01, 0.0, c01 )
        laff.axpy!( rho11, u01, c01 )
        laff.dots!( rho11, upsilon11, gamma11 )

        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        RTL, RTR, 
        RBL, RBR  = flame.cont_with_3x3_to_2x2(R00,  r01,   R02,  
                                               r10t, rho11, r12t, 
                                               R20,  r21,   R22,  
                                               "TL")

        CTL, CTR, 
        CBL, CBR  = flame.cont_with_3x3_to_2x2(C00,  c01,     C02,  
                                               c10t, gamma11, c12t, 
                                               C20,  c21,     C22,  
                                               "TL")

    end

    flame.merge_2x2!(CTL, CTR, 
                     CBL, CBR, C)

end

Trtrmm_uu_unb_var1! (generic function with 1 method)

In [12]:
C = copy( Cold ) # restore C 

Trtrmm_uu_unb_var1!( U, R, C )

using LinearAlgebra
strictly_lower_C = LowerTriangular( Cold ) - Diagonal(C)
Cref = UpperTriangular( U ) * UpperTriangular( R ) + strictly_lower_C
println( "C - Cref" )
C - Cref

C - Cref


5×5 Array{Float64,2}:
 0.366421  0.0       0.0       0.0       0.0     
 0.0       0.966634  0.212326  0.307637  0.173185
 0.0       0.0       0.35781   0.144311  0.587582
 0.0       0.0       0.0       0.940691  1.25537 
 0.0       0.0       0.0       0.0       0.639247

In theory, the result matrix should be (approximately) zero.

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.